## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import shutil
import os
import pandas as pd 
#
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN','NAME'],axis=1,inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
for col in application_df.columns:
    print(f'{col:<30}',len(application_df[col].unique()))

APPLICATION_TYPE               17
AFFILIATION                    6
CLASSIFICATION                 71
USE_CASE                       5
ORGANIZATION                   4
STATUS                         2
INCOME_AMT                     9
SPECIAL_CONSIDERATIONS         2
ASK_AMT                        8747
IS_SUCCESSFUL                  2


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
CUTOFF=520
application_types_to_replace=[]
for app in application_df['APPLICATION_TYPE'].unique():
    mask=application_df['APPLICATION_TYPE']==app
    cnt=application_df[mask]['APPLICATION_TYPE'].count()
    if cnt<CUTOFF:
        application_types_to_replace.append(app)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
col='CLASSIFICATION'
for v in application_df[col].unique():
    mask=application_df[col]==v
    cnt=application_df[mask][col].count()
    if cnt>1:
        print(f'{v:<30}',cnt)

C1000                          17326
C2000                          6074
C3000                          1918
C1200                          4837
C2700                          104
C7000                          777
C7200                          32
C1700                          287
C4000                          194
C7100                          75
C2800                          95
C6000                          15
C2100                          1883
C1238                          10
C5000                          116
C7120                          18
C1800                          15
C4100                          6
C1400                          34
C1270                          114
C2300                          32
C8200                          11
C1500                          16
C7210                          7
C1300                          58
C1230                          36
C1280                          50
C1240                          30
C2710                          3


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
CUTOFF2=1883
classifications_to_replace=[]
col='CLASSIFICATION'
for v in application_df[col].unique():
    mask=application_df[col]==v
    cnt=application_df[mask][col].count()
    if cnt<CUTOFF2:
        classifications_to_replace.append(v)
#         print(f'{v:<30}',cnt)
        
# # Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df=pd.get_dummies(application_df)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
y=application_df['IS_SUCCESSFUL']
application_df.drop('IS_SUCCESSFUL',axis=1,inplace=True)
X=application_df
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn=tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=43))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu", input_dim=45))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
804/804 [==============================] - 5s 3ms/step - loss: 0.5720 - accuracy: 0.7212
Epoch 2/10
804/804 [==============================] - 3s 3ms/step - loss: 0.5573 - accuracy: 0.7267
Epoch 3/10
804/804 [==============================] - 3s 4ms/step - loss: 0.5527 - accuracy: 0.7282
Epoch 4/10
804/804 [==============================] - 3s 4ms/step - loss: 0.5510 - accuracy: 0.7295
Epoch 5/10
804/804 [==============================] - 3s 4ms/step - loss: 0.5498 - accuracy: 0.7316
Epoch 6/10
804/804 [==============================] - 4s 5ms/step - loss: 0.5486 - accuracy: 0.7320
Epoch 7/10
804/804 [==============================] - 4s 5ms/step - loss: 0.5480 - accuracy: 0.7322
Epoch 8/10
804/804 [==============================] - 3s 4ms/step - loss: 0.5472 - accuracy: 0.7318
Epoch 9/10
804/804 [==============================] - 3s 4ms/step - loss: 0.5464 - accuracy: 0.7323
Epoch 10/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5459 - accuracy: 0.7325

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5581 - accuracy: 0.7311 - 686ms/epoch - 3ms/step
Loss: 0.5581008791923523, Accuracy: 0.7310787439346313


In [16]:
# Export our model to HDF5 file
nn.save_weights('model01.hdf5')

## Optimize Model

### Shrink the values some more

In [17]:
dirpath=('./untitled_project')
shutil.rmtree(dirpath)
os.mkdir(dirpath)

In [18]:
COLUMN_ADJUSTMENTS=[['APPLICATION_TYPE',800,'Other'],
                    ['CLASSIFICATION',2000,'Other'],
                   ]
COLUMNS_TO_DROP=['EIN','NAME','SPECIAL_CONSIDERATIONS','AFFILIATION']
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [19]:
def bin_small_items(col='CLASSIFICATION',cutoff=1883,desc='Other'):
    classifications_to_replace=[]
#     col='CLASSIFICATION'
    for v in application_df[col].unique():
        mask=application_df[col]==v
        cnt=application_df[mask][col].count()
        if cnt<cutoff:
            classifications_to_replace.append(v)
    #         print(f'{v:<30}',cnt)

    # # Replace in dataframe
    for cls in classifications_to_replace:
        application_df[col] = application_df[col].replace(cls,desc)    
    # Check to make sure binning was successful
    print(application_df[col].value_counts())
for spec in COLUMN_ADJUSTMENTS:
    col,cutoff,desc=spec
    bin_small_items(col,cutoff,desc)

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64
C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64


In [20]:
for col in application_df.columns:
    print(f'{col:<30}',len(application_df[col].unique()))

EIN                            34299
NAME                           19568
APPLICATION_TYPE               6
AFFILIATION                    6
CLASSIFICATION                 4
USE_CASE                       5
ORGANIZATION                   4
STATUS                         2
INCOME_AMT                     9
SPECIAL_CONSIDERATIONS         2
ASK_AMT                        8747
IS_SUCCESSFUL                  2


In [21]:
application_df.drop(COLUMNS_TO_DROP,axis=1,inplace=True)

C:\Users\rober\AppData\Local\Temp\ipykernel_16628\622044236.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  application_df.corr()


,STATUS,ASK_AMT,IS_SUCCESSFUL
STATUS,1.000000,0.000383,-0.001636
ASK_AMT,0.000383,1.000000,-0.010861
IS_SUCCESSFUL,-0.001636,-0.010861,1.000000


In [22]:
application_df=pd.get_dummies(application_df)
# Split our preprocessed data into our features and target arrays
y=application_df['IS_SUCCESSFUL']
application_df.drop('IS_SUCCESSFUL',axis=1,inplace=True)
X=application_df
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,stratify=y)
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# application_df.corr()
cor_matrix = application_df.corr().abs() 
print(cor_matrix)
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool)) 
print(upper_tri)
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)] 
print(to_drop)
application_df.drop(df.columns[to_drop],axis=1,inplace=True)
application_df.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,CLASSIFICATION_C1000,CLASSIFICATION_C1200,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
STATUS,1.000000,0.000383,0.003211,0.002162,-0.006258,0.002620,0.002272,0.002315,-0.002291,0.004893,...,0.002226,0.002958,0.001778,-0.017815,0.003988,0.001014,0.002043,-0.003513,0.000770,0.000889
ASK_AMT,0.000383,1.000000,-0.003635,-0.005657,0.012899,-0.006774,-0.004896,-0.005905,0.003932,-0.002140,...,0.008199,-0.049734,-0.004572,-0.003910,-0.009252,0.025872,0.001098,-0.009912,0.420248,0.006849
APPLICATION_TYPE_Other,0.003211,-0.003635,1.000000,-0.047612,-0.513195,-0.057706,-0.050049,-0.050991,0.255499,-0.107430,...,-0.246878,0.057429,-0.000078,0.000119,-0.051593,0.025549,-0.010054,-0.041219,0.005204,0.018874
APPLICATION_TYPE_T19,0.002162,-0.005657,-0.047612,1.000000,-0.345410,-0.038839,-0.033686,-0.034320,0.177180,-0.072534,...,-0.192297,0.084081,-0.008869,-0.015971,-0.039939,-0.015027,-0.029274,-0.048681,-0.011419,-0.013183
APPLICATION_TYPE_T3,-0.006258,0.012899,-0.513195,-0.345410,1.000000,-0.418641,-0.363091,-0.369927,-0.106859,0.207533,...,0.405456,-0.094375,-0.005379,0.008557,0.069341,0.007546,0.038692,0.042172,0.018456,0.006986
APPLICATION_TYPE_T4,0.002620,-0.006774,-0.057706,-0.038839,-0.418641,1.000000,-0.040828,-0.041596,-0.087223,-0.087103,...,-0.184616,0.056053,-0.002664,-0.004973,-0.037176,-0.016525,-0.027311,-0.017346,-0.013840,-0.015978
APPLICATION_TYPE_T5,0.002272,-0.004896,-0.050049,-0.033686,-0.363091,-0.040828,1.000000,-0.036077,-0.091939,-0.075325,...,-0.182507,0.008830,0.004567,-0.005874,-0.009366,-0.002324,-0.000644,-0.000074,-0.009479,-0.002906
APPLICATION_TYPE_T6,0.002315,-0.005905,-0.050991,-0.034320,-0.369927,-0.041596,-0.036077,1.000000,-0.085224,-0.074511,...,0.002487,-0.019000,0.018804,0.007262,0.004437,-0.016094,-0.013280,0.027372,-0.012230,-0.007661
CLASSIFICATION_C1000,-0.002291,0.003932,0.255499,0.177180,-0.106859,-0.087223,-0.091939,-0.085224,1.000000,-0.409380,...,0.072659,-0.082142,0.018309,0.006870,0.042615,0.026419,0.038842,0.030324,0.026421,0.017952
CLASSIFICATION_C1200,0.004893,-0.002140,-0.107430,-0.072534,0.207533,-0.087103,-0.075325,-0.074511,-0.409380,1.000000,...,-0.052178,0.059081,-0.007942,0.001627,-0.029764,-0.009895,-0.032931,-0.029651,-0.011343,-0.010396


### Run the optimizer

In [24]:
# Create a method that creates a new Sequential model with hyperparameter options
EPOCHS=5
ITERATIONS=3
# INPUT_DIM=42
INPUT_DIM=X_train_scaled.shape[1]
MAX_LAYERS=10
MAX_NODES=20

def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    activation = hp.Choice('activation',['relu','tanh','sigmoid','softmax'])
    hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam'])
    hp_lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    if hp_optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=hp_lr)
    elif hp_optimizer == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp_lr)
    elif hp_optimizer == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=hp_lr)
    else:
        raise
    
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=MAX_NODES,
        step=2), activation=activation, input_dim=INPUT_DIM))
    for i in range(hp.Int('num_layers', 1, MAX_LAYERS)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=MAX_NODES,
            step=2),
            activation=activation))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return nn_model

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=EPOCHS,
    hyperband_iterations=ITERATIONS)
print(X_train_scaled.shape,y_train.shape,X_test_scaled.shape,y_test.shape)

(25724, 30) (25724,) (8575, 30) (8575,)


In [25]:
tuner.search(X_train_scaled,y_train,epochs=EPOCHS,validation_data=(X_test_scaled,y_test))

Trial 30 Complete [00h 00m 11s]
val_accuracy: 0.6352186799049377

Best val_accuracy So Far: 0.6377842426300049
Total elapsed time: 00h 04m 31s
INFO:tensorflow:Oracle triggered exit


In [26]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6383 - accuracy: 0.6378 - 558ms/epoch - 2ms/step
Loss: 0.638258159160614, Accuracy: 0.6377842426300049
